# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import configparser
import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col,length
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format
from pyspark.sql import SQLContext

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [5]:
config = configparser.ConfigParser()
config.read('dl.cfg')
AWS_ACCESS_KEY_ID     = config.get('AWS', 'AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = config.get('AWS', 'AWS_SECRET_ACCESS_KEY')

In [6]:
spark = SparkSession \
            .builder\
            .config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
            .config("org.apache.hadoop:hadoop-aws:2.7.0")\
            .enableHiveSupport()\
            .getOrCreate()

### Analyzing USA demographics file us-cities-demographics.csv

In [41]:
demographics_file_nm = "us-cities-demographics.csv"

In [42]:
df = spark.read.format("csv").option("header", "true").option("delimiter",";").load(demographics_file_nm)

In [18]:
df.limit(5).toPandas()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.6,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129,49500,93629,4147,32935,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040,46799,84839,4819,8229,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127,87105,175232,5821,33878,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040,143873,281913,5829,86253,2.73,NJ,White,76402


In [12]:
print(df.count())

2891


### Analyzing Global Temprature file GlobalLandTemperaturesByCity.csv

In [13]:
temprature_file_path = "GlobalLandTemperaturesByCity.csv"

In [44]:
temp_df = spark.read.csv(temprature_file_path,header = True)

In [17]:
temp_df.limit(5).toPandas()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.7369999999999999,Århus,Denmark,57.05N,10.33E
1,1743-12-01,None,None,Århus,Denmark,57.05N,10.33E
2,1744-01-01,None,None,Århus,Denmark,57.05N,10.33E
3,1744-02-01,None,None,Århus,Denmark,57.05N,10.33E
4,1744-03-01,None,None,Århus,Denmark,57.05N,10.33E


In [19]:
print(temp_df.count())

8599212


### Analyzing airport file airport-codes_csv.csv

In [21]:
airport_file_path = 'airport-codes_csv.csv'
airport_df = spark.read.csv(airport_file_path,header = True)
airport_df.limit(5).toPandas()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,"-91.254898, 35.6087"


In [22]:
print(airport_df.count())

55075


### Analyzing data of USA immigration file data

In [23]:
immigration_file_nm = 'i94_apr16_sub.sas7bdat'
immigration_file_path = '../../data/18-83510-I94-Data-2016/'+immigration_file_nm
immigration_df = spark.read.format('com.github.saurfang.sas.spark').load(immigration_file_path)

In [24]:
immigration_df.limit(5).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,None,NaN,...,U,None,1979.0,10282016,None,None,None,1.897628e+09,None,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,None,1991.0,D/S,M,None,None,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,None,M,1961.0,09302016,M,None,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,None,M,1988.0,09302016,None,None,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,None,M,2012.0,09302016,None,None,AA,9.246846e+10,00199,B2


In [25]:
print(immigration_df.count())

3096313


### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

### Exploring and cleaning data issues of us-cities-demographics.csv file

In [43]:
record_count_before = df.count()
record_count_before

2891

In [27]:
df.printSchema()

root
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Median Age: string (nullable = true)
 |-- Male Population: string (nullable = true)
 |-- Female Population: string (nullable = true)
 |-- Total Population: string (nullable = true)
 |-- Number of Veterans: string (nullable = true)
 |-- Foreign-born: string (nullable = true)
 |-- Average Household Size: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- Race: string (nullable = true)
 |-- Count: string (nullable = true)



In [28]:
subset_columns = ["Median Age",
                      "Male Population",
                      "Female Population",
                      "Total Population",
                      "Number of Veterans",
                      "Foreign-born",
                      "Average Household Size",
                      "Count"]

In [29]:
df.dropna(subset = subset_columns,how = "all")

DataFrame[City: string, State: string, Median Age: string, Male Population: string, Female Population: string, Total Population: string, Number of Veterans: string, Foreign-born: string, Average Household Size: string, State Code: string, Race: string, Count: string]

In [30]:
df.drop_duplicates(subset = ["City","State","State Code","Race"])

DataFrame[City: string, State: string, Median Age: string, Male Population: string, Female Population: string, Total Population: string, Number of Veterans: string, Foreign-born: string, Average Household Size: string, State Code: string, Race: string, Count: string]

In [31]:
print("Number of records dropped after demographics file cleaning are {}".format(record_count_before - df.count()))

Number of records dropped after demographics file cleaning are 0


### Exploring and cleaning data issues of GlobalLandTemperaturesByCity.csv file

In [45]:
record_count_before = temp_df.count()
record_count_before

8599212

In [39]:
temp_df.printSchema()

root
 |-- dt: string (nullable = true)
 |-- AverageTemperature: string (nullable = true)
 |-- AverageTemperatureUncertainty: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [35]:
temp_df = temp_df.filter("AverageTemperature is not null")

In [36]:
temp_df = temp_df.drop_duplicates(subset = ["dt","City","Country"])

In [37]:
print("Number of records dropped after temprature file cleaning are {}".format(record_count_before - temp_df.count()))

Number of records dropped after temprature file cleaning are 408429


### Exploring and cleaning data issues of airport-codes_csv.csv file

In [46]:
record_count_before = airport_df.count()
record_count_before

55075

In [47]:
airport_df.printSchema()

root
 |-- ident: string (nullable = true)
 |-- type: string (nullable = true)
 |-- name: string (nullable = true)
 |-- elevation_ft: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- iso_country: string (nullable = true)
 |-- iso_region: string (nullable = true)
 |-- municipality: string (nullable = true)
 |-- gps_code: string (nullable = true)
 |-- iata_code: string (nullable = true)
 |-- local_code: string (nullable = true)
 |-- coordinates: string (nullable = true)



In [49]:
airport_df = airport_df.withColumn('length_column', length('iata_code'))
airport_df = airport_df.filter(col('length_column') == 3)

In [50]:
airport_df = airport_df.select(['iata_code','type','name','elevation_ft','continent','iso_country','iso_region','municipality','coordinates'])

In [51]:
airport_df = airport_df.drop_duplicates(subset = ["iata_code"])

In [52]:
print("Number of records dropped after airport file cleaning are {}".format(record_count_before - airport_df.count()))

Number of records dropped after airport file cleaning are 46035


### Exploring and cleaning USA immigration data arrival date, visa, fact table

In [53]:
record_count_before = immigration_df.count()
record_count_before

3096313

In [57]:
immigration_df.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [54]:
arrival_dt_df = immigration_df.drop_duplicates(subset = ["arrdate"])

In [55]:
arrival_dt_df = arrival_dt_df.dropna(subset = ["arrdate"],how = "all")

In [56]:
arrival_dt_df = arrival_dt_df.select(["arrdate"])

In [58]:
arrival_dt_df = arrival_dt_df.distinct()
get_timestamp = udf(lambda x: (datetime.datetime(1960, 1, 1).date() + datetime.timedelta(x)).isoformat() if x else None)
arrival_dt_df = arrival_dt_df.select(['arrdate']).withColumn("arr_date", get_timestamp(arrival_dt_df.arrdate))

In [59]:
print("Number of records dropped after arrival date dataset cleaning are {}".format(record_count_before - arrival_dt_df.count()))

Number of records dropped after arrival date dataset cleaning are 3096283


In [60]:
visa_df = immigration_df.drop_duplicates(subset = ["visatype"])

In [61]:
visa_df = visa_df.dropna(subset = ["visatype"],how = "all")

In [62]:
visa_df = visa_df.select(["visatype"])
visa_df = visa_df.distinct()

In [63]:
print("Number of records dropped after visa type dataset cleaning are {}".format(record_count_before - visa_df.count()))

Number of records dropped after visa type dataset cleaning are 3096296


In [64]:
fact_df = immigration_df.drop_duplicates(subset = ["cicid"])

In [65]:
fact_df = fact_df.dropna(how = "all")
fact_df = fact_df.select(["cicid","visatype","i94res","i94port","arrdate",
                    "i94mode","i94addr","depdate","i94bir","count",
                    "biryear","gender","airline","fltno"])
fact_df = fact_df.filter("""cicid is not null and
                      visatype is not null and
                      i94res is not null and
                      i94port is not null and
                      arrdate is not null and
                      i94mode is not null and
                      i94addr is not null and
                      depdate is not null and
                      i94bir is not null and
                      count is not null and
                      biryear is not null and
                      gender is not null and
                      airline is not null and
                      fltno is not null""")

In [66]:
print("Number of records dropped after fact table dataset cleaning are {}".format(record_count_before - fact_df.count()))

Number of records dropped after fact table dataset cleaning are 711668


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.